In [2]:
import os 
import pandas as pd 
import numpy as np
import collections

import warnings
warnings.filterwarnings("ignore")

In [3]:
# set working directory (Change for youe)
os.chdir('C:/Users/ander/Google Drive/Columbia/Fall 2019/Capstone/Dotin-Columbia-Castone-Team-Alpha-')

# initilize path
mouse_flat_path = 'Data/Clean Data/mouse_flat_v4.csv'

# read file 
data = pd.read_csv(mouse_flat_path)

FileNotFoundError: [Errno 2] File b'Data/Clean Data/mouse_flat_v4.csv' does not exist: b'Data/Clean Data/mouse_flat_v4.csv'

In [3]:
direction = data[['user_id', 'direction']].values
user_list = set(data.user_id)
page_movement = collections.deque()
for user in user_list: 
    user_segment = direction[direction[:,0] == user]
    count = 1
    for user, action in user_segment:
        if action == 'PageChange ,':
            count += 1
            user_movement = f'page {count}'
            page_movement.append(user_movement)
        else: 
            user_movement = f'page {count}'
            page_movement.append(user_movement)   
            
data.direction = page_movement

In [ ]:
# filter users with less than 196 radio count, which is the number of questions in the data 
subset_radio = data.loc[:,['user_id', 'radio']]
subset_radio = subset_radio.dropna(subset = ['radio'])
user_completion = subset_radio.groupby('user_id').count() >= 196
user_completion = user_completion[user_completion['radio'] == True].reset_index()
user_id_who_completed_survey = pd.DataFrame(user_completion['user_id'])
completed_survey_data = data[data.user_id.isin(user_id_who_completed_survey.user_id)].reset_index()
completed_survey_data = completed_survey_data.drop(columns = ['Index','index'])

In [4]:
len(set(completed_survey_data.user_id))

NameError: name 'completed_survey_data' is not defined

In [5]:
coordinates =  data[['user_id','cord_x', 'cord_y','action', 'direction']].to_numpy()

In [6]:
def parse_distance(coord_data):
    """
    Calculate the incremental coordinate for each mouse movement
    :param coord_data:
    :return: A seperate numpy array for the x, y, user id list.
    """
    user_id = np.unique(coord_data[:, 0])[1:]
    total_x = []
    total_y = []
    user_id_list = []
    action_list = []
    page_list = []

    for user in user_id:
        index = coord_data[:, 0] == user
        new_index = coord_data[index]

        id_updated = new_index[1:, 0]
        action_updated =new_index[1:,3]
        page_updated =new_index[1:,4]

        distance_x = new_index[1:, 1] - new_index[:len(new_index) - 1, 1]
        distance_y = new_index[1:, 2] - new_index[:len(new_index) - 1, 2]

        total_x = np.append([total_x], [distance_x])
        total_y = np.append([total_y], [distance_y])
        
        user_id_list = np.append([user_id_list], [id_updated])
        action_list = np.append([action_list], [action_updated])
        page_list = np.append([page_list], [page_updated])

    return total_x, total_y, user_id_list, action_list, page_list


def parse_directions(id_list, x_coord, y_coord, action):
    """
    Converts coordinate changes all mouse movements into Cardinal direction. Ignores scrolls and clicks 

    :param id_list: Unique list of user ids
    :param x_coord: List of the changes in the x coordinate
    :param y_coord: List of the changes in the y coordinates
    :return: A data frame that contains the user id list, x directional changes, and y directional changes,
    calculated cardinal direction
    """

    directions = collections.deque()  # optimized for append operations
    
    for value in range(0, len(id_list)):

        x = x_coord[value]
        y = y_coord[value]
        
        action_value = action[value]
        
        if action_value == 'm':
            if x == 0 and y == 0:
                directions.append('No Movement')
            elif x > 0 and y == 0:
                directions.append('East')
            elif x < 0 and y == 0:
                directions.append('West')
            elif x == 0 and y > 0:
                directions.append('North')
            elif x == 0 and y < 0:
                directions.append('South')
            elif x > 0 and y > 0:
                directions.append('North East')
            elif x > 0 and y < 0:
                directions.append('North West')
            elif x < 0 and y < 0:
                directions.append('South West')
            elif x < 0 and y > 0:
                directions.append('South East')
            else:
                directions.append('TBD')  
        else: 
            directions.append(action_value)

    return pd.DataFrame({'User Id': id_list,
                         'Distance X': x_coord,
                         'Distance Y': y_coord,
                         'Direction': directions})

In [7]:
# parse distance
coord_x, coord_y, coord_user, action_type, page_list = parse_distance(coordinates)

In [8]:
user_sytem = data[['user_id', 'system']].drop_duplicates()
user_sytem['system'].value_counts()

pc         718
Android      4
unknown      4
IpadOld      3
Windows      1
Name: system, dtype: int64

In [18]:
# parse direction
direction_data = parse_directions(coord_user, coord_x, coord_y, action_type)
direction_data['Page count'] = page_list
direction_data['Direction'].value_counts()

No Movement    1304009
East            959081
North East      934562
North           934043
South East      840150
West            822364
South West      749988
North West      580863
South           551432
c               235556
s               205113
np                 151
Name: Direction, dtype: int64

In [19]:
# remove all clicks, scrolls, and np 
direction_data = direction_data[-direction_data.Direction.isin(['c','s','np'])]
direction_data['Direction'].value_counts()

No Movement    1304009
East            959081
North East      934562
North           934043
South East      840150
West            822364
South West      749988
North West      580863
South           551432
Name: Direction, dtype: int64

In [11]:
class page_description:
    """
    Class to analyze direction data. Purpose is to determine the movement count thresholdhold for hmm model.  
    """
    def __init__(self, page_number):
        self.page_number = page_number
        self.page = direction_data[direction_data['Page count'] == f'page {self.page_number}']

    def page_segment(self):
        """
        Desplays the user id belong to a that segment
        """
        page = direction_data[direction_data['Page count'] == f'page {self.page_number}']
        page = page.groupby('User Id').count()[['Page count']]
        return page

    def page_description(self):
        """
        Provide summary statistics on a page 
        """
        
        return self.page_segment().describe()

    def percentage(self):
        """
        prints the percentage of the total number of counts recorded on this page
        """
        
        value = f' {round((len(self.page) / len(direction_data)) * 100,2)}% of all the mouse movemens are recorded in page {self.page_number}'
        
        return value

Page 1 

In [82]:
page_description(1).percentage()

' 38.1% of all the mouse movemens are recorded in page 1'

In [83]:
page_description(1).page_description()

,Page count
count,533.000000
mean,4115.930582
std,5201.652347
min,2.000000
25%,396.000000
50%,1136.000000
75%,7699.000000
max,25945.000000


Page 2

In [84]:
page_description(2).percentage()

' 7.91% of all the mouse movemens are recorded in page 2'

In [85]:
page_description(2).page_description()

,Page count
count,346.000000
mean,1317.315029
std,1256.536644
min,2.000000
25%,534.500000
50%,988.000000
75%,1693.000000
max,12952.000000


Page 3

In [86]:
page_description(3).percentage()

' 7.44% of all the mouse movemens are recorded in page 3'

In [87]:
page_description(3).page_description()

,Page count
count,336.000000
mean,1274.398810
std,1620.941087
min,5.000000
25%,456.500000
50%,869.000000
75%,1553.750000
max,16887.000000


Page 4 

In [88]:
page_description(4).percentage()

' 6.21% of all the mouse movemens are recorded in page 4'

In [89]:
page_description(4).page_description()

,Page count
count,323.000000
mean,1106.873065
std,1227.339345
min,2.000000
25%,424.500000
50%,774.000000
75%,1435.500000
max,15648.000000


Page 5 

In [90]:
page_description(5).percentage()

' 5.83% of all the mouse movemens are recorded in page 5'

In [91]:
page_description(5).page_description()

,Page count
count,317.000000
mean,1058.384858
std,1133.153197
min,2.000000
25%,417.000000
50%,738.000000
75%,1395.000000
max,13659.000000


Page 6

In [92]:
page_description(6).percentage()

' 5.04% of all the mouse movemens are recorded in page 6'

In [93]:
page_description(6).page_description()

,Page count
count,309.000000
mean,938.624595
std,1144.467018
min,3.000000
25%,373.000000
50%,670.000000
75%,1119.000000
max,14736.000000


Page 7

In [94]:
page_description(7).percentage()

' 4.87% of all the mouse movemens are recorded in page 7'

In [95]:
page_description(7).page_description()

,Page count
count,303.000000
mean,926.455446
std,1116.360701
min,5.000000
25%,374.500000
50%,656.000000
75%,1174.000000
max,14516.000000


Page 8 

In [96]:
page_description(8).percentage()

' 4.33% of all the mouse movemens are recorded in page 8'

In [97]:
page_description(8).page_description()

,Page count
count,281.000000
mean,887.725979
std,861.240053
min,2.000000
25%,386.000000
50%,653.000000
75%,1162.000000
max,9498.000000


Page 9 

In [98]:
page_description(9).percentage()

' 4.02% of all the mouse movemens are recorded in page 9'

In [99]:
page_description(9).page_description()

,Page count
count,268.000000
mean,864.652985
std,899.286814
min,1.000000
25%,347.750000
50%,652.000000
75%,1148.000000
max,10797.000000


Page 10 

In [100]:
page_description(10).percentage()

' 3.68% of all the mouse movemens are recorded in page 10'

In [101]:
page_description(10).page_description()

,Page count
count,258.000000
mean,820.883721
std,1206.678156
min,1.000000
25%,279.500000
50%,584.500000
75%,1042.500000
max,14741.000000


Page 11

In [102]:
page_description(11).percentage()

' 3.36% of all the mouse movemens are recorded in page 11'

In [103]:
page_description(11).page_description()

,Page count
count,230.000000
mean,842.121739
std,829.786016
min,2.000000
25%,301.250000
50%,591.500000
75%,1119.500000
max,6122.000000


Page 12

In [104]:
page_description(12).percentage()

' 2.8% of all the mouse movemens are recorded in page 12'

In [105]:
page_description(12).page_description()

,Page count
count,223.000000
mean,724.264574
std,740.030592
min,3.000000
25%,309.500000
50%,556.000000
75%,978.000000
max,7856.000000


Page 13

In [106]:
page_description(13).percentage()

' 2.49% of all the mouse movemens are recorded in page 13'

In [107]:
page_description(13).page_description()

,Page count
count,193.000000
mean,743.580311
std,948.819966
min,3.000000
25%,284.000000
50%,555.000000
75%,903.000000
max,9330.000000


Page 14

In [108]:
page_description(14).percentage()

' 2.21% of all the mouse movemens are recorded in page 14'

In [109]:
page_description(14).page_description()

,Page count
count,177.000000
mean,718.344633
std,718.100683
min,2.000000
25%,268.000000
50%,545.000000
75%,951.000000
max,5285.000000


Page 15

In [110]:
page_description(15).percentage()

' 1.71% of all the mouse movemens are recorded in page 15'

In [111]:
page_description(15).page_description()

,Page count
count,170.000000
mean,577.847059
std,557.222954
min,3.000000
25%,217.500000
50%,458.000000
75%,854.750000
max,5093.000000


In [13]:
observation =  collections.deque()
for user in user_list: 
    user_subset = direction_data[direction_data['User Id'] == user]
    obs_count = 1
    for row in range(0, len(user_subset)):
        observation.append(obs_count)
        obs_count += 1

In [28]:
direction_data['observation'] = observation

In [29]:
direction_data

,User Id,Distance X,Distance Y,Direction,Page count,obsersvation,observation
0,371,707,224,North East,page 1,1,1
1,371,4,-20,North West,page 1,2,2
2,371,0,-30,South,page 1,3,3
3,371,0,-18,South,page 1,4,4
4,371,1,-2,North West,page 1,5,5
...,...,...,...,...,...,...,...
8117306,1838,-3,0,West,page 1,13014,13014
8117307,1838,-2,0,West,page 1,13015,13015
8117308,1838,-2,0,West,page 1,13016,13016
8117309,1838,-1,0,West,page 1,13017,13017


In [27]:
page_description(15).page_description().iloc[4]

Page count    217.5
Name: 25%, dtype: float64

In [40]:
direction_data['Direction'].isin(page_number_list)

In [48]:
direction_data = direction_data[direction_data['Page count'].isin(page_number_list)]
direction_data

,User Id,Distance X,Distance Y,Direction,Page count,obsersvation,observation
0,371,707,224,North East,page 1,1,1
1,371,4,-20,North West,page 1,2,2
2,371,0,-30,South,page 1,3,3
3,371,0,-18,South,page 1,4,4
4,371,1,-2,North West,page 1,5,5
...,...,...,...,...,...,...,...
8117306,1838,-3,0,West,page 1,13014,13014
8117307,1838,-2,0,West,page 1,13015,13015
8117308,1838,-2,0,West,page 1,13016,13016
8117309,1838,-1,0,West,page 1,13017,13017


In [41]:
page_number_list =[]
for value in range(1,16):
    page_ = f'page {value}'
    page_number_list.append(page_)

In [55]:
subset = direction_data[direction_data['Page count'] == page_number_list[0]]

In [114]:
observation_limit = 300

In [117]:
page_1_ = subset[subset['observation'] <= observation_limit]